In [181]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import sklearn
import math
from prettytable import PrettyTable
from functools import partial

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='Microsoft YaHei', font_scale=1.2,
        palette=palette)  # 解决Seaborn中文显示问题


def toAngle(x): return x * (math.pi / 180)

In [182]:
alpha = 1.5
theta = 120

In [183]:
def 计算海水深度(x):
    return (x, 70-x * math.tan(toAngle(alpha)))


def 计算覆盖宽度(距离测线中心点的距离, 海水深度):
    val = (海水深度 * math.cos(toAngle(alpha)) * math.sin(toAngle(theta))) / \
        (math.cos(toAngle(alpha - theta / 2))
         * math.cos(toAngle(alpha + theta / 2)))
    return (距离测线中心点的距离,
            val)


def 计算重叠率(距离测线中心点的距离, 海水深度, 覆盖宽度):
    d = 200
    val = 1 - (d / 覆盖宽度)
    return (距离测线中心点的距离, val)


def 计算重叠率2(距离测线中心点的距离, 深度列表, 覆盖宽度):
    print(距离测线中心点的距离, 深度列表, 覆盖宽度)
    前一个深度, 当前深度 = 深度列表
    if 前一个深度 is None:
        return (前一个深度, None)
    d = 200
    
    val = math.sin(toAngle(theta / 2)) \
        * (前一个深度 / math.cos(toAngle(alpha - theta/2))
            + 当前深度 / math.cos(toAngle(alpha + theta/2))) \
        - d / math.cos(toAngle(alpha)) \

    val = val / 覆盖宽度
    return (距离测线中心点的距离, val)

In [184]:
距离测线中心点的距离 = [-800, -600, -400, -200, 0, 200, 400, 600, 800]
海水深度 = dict(map(计算海水深度, 距离测线中心点的距离))
覆盖宽度 = dict(map(计算覆盖宽度, 距离测线中心点的距离, 海水深度.values()))
重叠率 = dict(map(计算重叠率, 距离测线中心点的距离, 海水深度.values(), 覆盖宽度.values()))


def 转二元组(x):
    tmp = [None] + list(x)
    result = []
    for idx, val in enumerate(tmp):
        if val is None:
            continue
        result.append((tmp[idx - 1], val))

    return result

重叠率2 = dict(map(计算重叠率2, 距离测线中心点的距离, 转二元组(海水深度.values()), 覆盖宽度.values()))

-800 (None, 90.94873725534954) 315.8133282933295
-600 (90.94873725534954, 85.71155294151215) 297.62756058558364
-400 (85.71155294151215, 80.47436862767478) 279.44179287783777
-200 (80.47436862767478, 75.23718431383739) 261.25602517009185
0 (75.23718431383739, 70.0) 243.07025746234592
200 (70.0, 64.76281568616261) 224.88448975460003
400 (64.76281568616261, 59.52563137232523) 206.69872204685413
600 (59.52563137232523, 54.28844705848784) 188.51295433910823
800 (54.28844705848784, 49.05126274465046) 170.32718663136234


In [185]:
data = {
    '距离测线中心点的距离': 距离测线中心点的距离,
    '海水深度': 海水深度.values(),
    '覆盖宽度': 覆盖宽度.values(),
    '重叠率': 重叠率.values(),
    '重叠率2': 重叠率2.values(),
}
data

{'距离测线中心点的距离': [-800, -600, -400, -200, 0, 200, 400, 600, 800],
 '海水深度': dict_values([90.94873725534954, 85.71155294151215, 80.47436862767478, 75.23718431383739, 70.0, 64.76281568616261, 59.52563137232523, 54.28844705848784, 49.05126274465046]),
 '覆盖宽度': dict_values([315.8133282933295, 297.62756058558364, 279.44179287783777, 261.25602517009185, 243.07025746234592, 224.88448975460003, 206.69872204685413, 188.51295433910823, 170.32718663136234]),
 '重叠率': dict_values([0.36671450479683787, 0.3280192210476105, 0.284287443405314, 0.23446741613025324, 0.17719262698776683, 0.11065453994517205, 0.032408144474815215, -0.06093504661874949, -0.1742106703896793]),
 '重叠率2': dict_values([None, 0.3569544259727246, 0.3151057199709473, 0.2674309217615544, 0.21262236054223005, 0.1489493750683201, 0.07407223591972305, -0.01525163728968, -0.12364966115077022])}

In [186]:
df = pd.DataFrame(data)
df.T

,0,1,2,3,4,5,6,7,8
距离测线中心点的距离,-800.000000,-600.000000,-400.000000,-200.000000,0.000000,200.000000,400.000000,600.000000,800.000000
海水深度,90.948737,85.711553,80.474369,75.237184,70.000000,64.762816,59.525631,54.288447,49.051263
覆盖宽度,315.813328,297.627561,279.441793,261.256025,243.070257,224.884490,206.698722,188.512954,170.327187
重叠率,0.366715,0.328019,0.284287,0.234467,0.177193,0.110655,0.032408,-0.060935,-0.174211
重叠率2,NaN,0.356954,0.315106,0.267431,0.212622,0.148949,0.074072,-0.015252,-0.123650


In [187]:
df.T.to_excel('第一问ans.xlsx')